In [5]:
import pickle

fname = '/home/anton/Documents/Tulane/Hackathon/ImageNomeR/data/fmri-FC-epr-slim.pkl'
fmriDict = None

with open(fname, 'rb') as f:
    fmriDict = pickle.load(f)
    
print(fmriDict.keys())

dict_keys(['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap'])


In [6]:
def getGroupIdcs(groupsMap):
    normals = []
    fibros = []
    for num,diag in groupsMap.items():
        if diag == 1:
            normals.append(num)
        else:
            fibros.append(num)
    return normals,fibros

normals, fibros = getGroupIdcs(fmriDict['groupsNormalDiagMap'])

print(len(normals))
print(normals)
print(len(fibros))
print(fibros)

33
['007', '012', '014', '016', '018', '021', '022', '026', '030', '031', '032', '033', '034', '036', '042', '045', '047', '056', '058', '059', '060', '061', '064', '066', '068', '069', '070', '072', '073', '074', '075', '076', '077']
32
['002', '004', '006', '008', '009', '010', '011', '013', '015', '017', '019', '020', '023', '024', '025', '028', '029', '037', '038', '039', '040', '043', '044', '046', '049', '050', '052', '053', '054', '055', '062', '063']


In [15]:
nRuns = 50
model = 'Logistic Regression L2 C=1'
desc = 'normal(1) vs. fibromyalgia(0) epr fMRI only'

import random
import numpy as np

from sklearn.linear_model import LogisticRegression
import sys

sys.path.append('/home/anton/Documents/Tulane/Hackathon/ImageNomeR/src')

from imagenomer import Analysis, JsonData, JsonSubjects, JsonFCMetadata

analysis = Analysis(f'{desc}: {model}', 'localhost')

a,b = np.triu_indices(264,1)
idcs = np.arange(34716)

labels = [f'{a[i]}-{b[i]}' for i in idcs]

print(labels[0:10])
print(labels[-10:])

for run in range(nRuns):
    FCslim = fmriDict['FC-slim']
    subjNum2Idx = fmriDict['subjNum2IdxMap']

    random.shuffle(normals)
    random.shuffle(fibros)

    normalTrain = normals[:25]
    normalTest = normals[25:]

    fibrosTrain = fibros[:25]
    fibrosTest = fibros[25:]

    train = normalTrain + fibrosTrain
    test = normalTest + fibrosTest

    Xtrain = [FCslim[subjNum2Idx[num]] for num in train]
    Xtest = [FCslim[subjNum2Idx[num]] for num in test]

    Xtrain = np.stack(Xtrain)
    Xtest = np.stack(Xtest)

    ytrain = np.concatenate([np.ones(25), np.zeros(25)])
    ytest = np.concatenate([np.ones(8), np.zeros(7)])

#     print(Xtrain.shape)
#     print(Xtest.shape)
#     print(ytrain.shape)
#     print(ytest.shape)

#     clf = LogisticRegression(max_iter=5000, C=1e-5).fit(Xtrain, ytrain)
    clf = LogisticRegression(max_iter=5000, C=0.001, 
                             solver='saga', penalty='l1').fit(Xtrain, ytrain)
    yhat = clf.predict(Xtest)
    acc = sum(yhat == ytest)/len(ytest)
    
    print(f'{run}. {acc}')
    
    jsonCompare = desc
    jsonAccuracy = acc
    jsonTrain = [25,25]
    jsonTest = [8,8]
    jsonWeights = clf.coef_.squeeze()
    jsonLabels = labels

    jsonObj = {
        'Compare': jsonCompare,
        'Model': 'Logistic Regression',
        'Accuracy': jsonAccuracy,
        'Train': jsonTrain,
        'Test': jsonTest,
        'Weights': list(jsonWeights),
        'Labels': jsonLabels
    }
    
    dat = JsonData(analysis)
    dat.dict.update(jsonObj)
    r = dat.post()
    print(r.content)


['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10']
['259-260', '259-261', '259-262', '259-263', '260-261', '260-262', '260-263', '261-262', '261-263', '262-263']
0. 0.4666666666666667
b'Success'
1. 0.4666666666666667
b'Success'
2. 0.5333333333333333
b'Success'
3. 0.4666666666666667
b'Success'
4. 0.4666666666666667
b'Success'
5. 0.5333333333333333
b'Success'
6. 0.5333333333333333
b'Success'
7. 0.4666666666666667
b'Success'
8. 0.4666666666666667
b'Success'
9. 0.5333333333333333
b'Success'
10. 0.4666666666666667
b'Success'
11. 0.4666666666666667
b'Success'
12. 0.5333333333333333
b'Success'
13. 0.5333333333333333
b'Success'
14. 0.5333333333333333
b'Success'
15. 0.4666666666666667
b'Success'
16. 0.4666666666666667
b'Success'
17. 0.4666666666666667
b'Success'
18. 0.5333333333333333
b'Success'
19. 0.5333333333333333
b'Success'
20. 0.4666666666666667
b'Success'
21. 0.4666666666666667
b'Success'
22. 0.5333333333333333
b'Success'
23. 0.4666666666666667
b'Success'
24. 0.46666